In [3]:
from PIL import Image, ImageDraw
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

# Load the model
model_save_path = r"C:\\Users\\thath\\OneDrive - University of Bradford\\Discipline-specific Artificial Intelligence Project\\Project file\\Test 02\\model.keras"
model = load_model(model_save_path)

# Load the label_to_index dictionary
label_to_index_save_path = r"C:\\Users\\thath\\OneDrive - University of Bradford\\Discipline-specific Artificial Intelligence Project\\Project file\\Test 02\\Testing code\\label_to_index.pkl"
with open(label_to_index_save_path, 'rb') as f:
    label_to_index = pickle.load(f)

# Create a reverse mapping from index to label
index_to_label = {index: label for label, index in label_to_index.items()}

# Define colors for each category
category_colors = {
    "AnnualCrop": (255, 255, 0, 128),  # Yellow with transparency
    "Forest": (0, 128, 0, 128),  # Green with transparency
    "HerbaceousVegetation": (144, 238, 144, 128),  # Light Green with transparency
    "Industrial": (255, 165, 0, 128),  # Orange with transparency
    "Pasture": (173, 216, 230, 128),  # Light Blue with transparency
    "PermanentCrop": (0, 100, 0, 128),  # Dark Green with transparency
    "Residential": (255, 0, 0, 128),  # Red with transparency
    "River": (0, 0, 255, 128),  # Blue with transparency
    "SeaLake": (0, 255, 255, 128)  # Cyan with transparency
}

# Preprocess the input image
def preprocess_image(image):
    img = image.resize((64, 64))  # Resize image to match the input size of the model
    img_array = np.array(img) / 255.0  # Normalize image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Process the larger image and highlight the smaller images
def process_large_image(image_path):
    original_image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB mode
    width, height = original_image.size
    final_image = original_image.copy()
    draw = ImageDraw.Draw(final_image, 'RGBA')

    category_counts = {label: 0 for label in index_to_label.values()}

    for i in range(0, width, 64):
        for j in range(0, height, 64):
            box = (i, j, i + 64, j + 64)
            cropped_image = original_image.crop(box)
            preprocessed_image = preprocess_image(cropped_image)
            predictions = model.predict(preprocessed_image)
            predicted_class = np.argmax(predictions, axis=1)[0]
            predicted_label = index_to_label[predicted_class]

            # Count the occurrences of each category
            category_counts[predicted_label] += 1

            # Debugging: Print the predicted label and corresponding color
            color = category_colors.get(predicted_label, (255, 255, 255, 128))  # Default to semi-transparent white if category not found
            print(f"Box {box}: {predicted_label}, Color: {color}")

            # Highlight the image according to its category
            draw.rectangle(box, fill=color)

    return final_image, category_counts

# Calculate and display category changes
def calculate_category_changes(counts1, counts2):
    total1 = sum(counts1.values())
    total2 = sum(counts2.values())

    changes = {}
    for category in counts1:
        percentage1 = (counts1[category] / total1) * 100 if total1 > 0 else 0
        percentage2 = (counts2[category] / total2) * 100 if total2 > 0 else 0
        changes[category] = (percentage1, percentage2, percentage2 - percentage1)

    return changes

# Display the predicted images side by side and show category changes
def display_predicted_images_and_changes(image_path1, image_path2):
    predicted_image1, counts1 = process_large_image(image_path1)
    predicted_image2, counts2 = process_large_image(image_path2)

    # Get dimensions
    width1, height1 = predicted_image1.size
    width2, height2 = predicted_image2.size

    # Create a new image with width equal to the sum of both images' widths and height equal to the maximum height
    combined_width = width1 + width2
    combined_height = max(height1, height2)
    combined_image = Image.new('RGB', (combined_width, combined_height))

    # Paste the images side by side
    combined_image.paste(predicted_image1, (0, 0))
    combined_image.paste(predicted_image2, (width1, 0))

    # Display the combined image
    combined_image.show()

    # Calculate and display category changes
    changes = calculate_category_changes(counts1, counts2)
    print(changes)
    for category, (percentage1, percentage2, change) in changes.items():
        print(f"{category}: {percentage1:.2f}% -> {percentage2:.2f}% (Change: {change:.2f}%)")

# Paths to your images
image_path1 = r"C:\\Users\\thath\\OneDrive - University of Bradford\\Discipline-specific Artificial Intelligence Project\\Test sampel\\1.2.jpg"
image_path2 = r"C:\\Users\\thath\\OneDrive - University of Bradford\\Discipline-specific Artificial Intelligence Project\\Test sampel\\1.1.jpg"

# Display the predicted images side by side and show category changes
display_predicted_images_and_changes(image_path1, image_path2)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 482ms/step
Box (0, 0, 64, 64): Forest, Color: (0, 128, 0, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Box (0, 64, 64, 128): Pasture, Color: (173, 216, 230, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
Box (0, 128, 64, 192): Pasture, Color: (173, 216, 230, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Box (0, 192, 64, 256): Pasture, Color: (173, 216, 230, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Box (0, 256, 64, 320): Pasture, Color: (173, 216, 230, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Box (0, 320, 64, 384): Pasture, Color: (173, 216, 230, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Box (0, 384, 64, 448): Forest, Color: (0, 128, 0, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Box (0, 448, 64, 512): Forest, Color: (0, 128, 0, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
Box (0, 512, 64, 576): Forest, Color: (0, 128, 0, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
Box (0, 576, 64, 640): Forest, Color: (0, 128, 0, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Box (0